# The problem statement-
Consider the following situation in a computer science laboratory. On any average day about 10 students are working in the lab
at any given hour. These students typically print up to twice during that time and the lengths of these tasks ranges from 
1 to 20 pages. The printer in the lab is older, capable of processing 10 pages per minute. The slower printing speed could
make students wait too long. What page rate should be used ?

# The solution approach-
Building a simulation that models the laboratory. We will need to construct representations for-
a) Students
b) Printing Tasks
c) Printer

As students submit printing tasks, we will add them to the waiting list, a queue of printing tasks attached to the printer.
When the printer completes a task, it will look at the queue to see if there are any remaining tasks to process.
Of interest for us is the average amount of time students will wait for their papers to be printed is equal to the average
amount of time a task waits in the queue.

Here, is the main simulation steps:

1)  Create a queue of print tasks. Each task will be given a timestamp upon its arrival. The queue is empty to start.

2)  For each second (currentSecond):
    
    a) Does a new print task get created ? If so, add it to the queue with the currentSecond as the timestamp
    
    b) If the printer is not busy and the if a task is waiting-
        i) Remove the next task from the print queue and assign it to the printer.
        ii) Subtract the timestamp from the currentSecond to compute the waiting time for that task.
        iii) Append the waiting time for that task to a list for later processing.
        iv) Based on the number of pages in the print task, figure out how much time will be required.
        
    c) The printer now does one second of printing if necessary. It also subtracts one second from the time required for that task.
    d) If the task has been completed, in other words the time required has reached zero, the printer is no longer busy.
3) After the simulation is complete, compute the average waiting time from the list of waiting times generated.

In [14]:
class myQueue:
    def __init__(self):
        self.items = []
    
    def __str__(self):
        return str(self.items)
    
    def enqueue(self, item):
        self.items.insert(0, item)
        
    def dequeue(self):
        return self.items.pop()
    
    def isEmpty(self):
        return self.items == []
    
    def size(self):
        return len(self.items)

In [15]:
class Printer:
    def __init__(self, ppm):
        self.pagerate = ppm  # passing a page rate to the printer
        self.currentTask = None  # initially no task assigned, it's idle
        self.timeRemaining = 0 # as its idle, so the timeRemaining to complete the job is also 0
        
    # This method defines the clock for the printer
    def tick(self):
        if self.currentTask != None:  # if the printer is busy, i.e. some task is in process
            self.timeRemaining = self.timeRemaining - 1 # decreasing by 1 second as it's our unit of time measure
            if self.timeRemaining <= 0:  # if job is done
                self.currentTask = None  # the current task is completed
                
    # Returns if the printer is busy or not
    def busy(self):
        if self.currentTask != None:
            return True
        else:
            return False
        
    # It takes the new task from the task queue
    def startNext(self, newtask):
        self.currentTask = newtask
        self.timeRemaining = newtask.getPages() * 60/self.pagerate  # this computes timeRemaining in seconds

In [16]:
import random

class Task:
    def __init__(self, time):
        self.timestamp = time  # when the task comes in the queue
        self.pages = random.randrange(1, 21)  # any number of pages between 1 and 20 could be given by the students
        
    def getStamp(self):
        return self.timestamp
    
    def getPages(self):
        return self.pages
    
    def waitTime(self, currenttime):
        return currenttime - self.timestamp # it calculates the waiting time in the queue. 'currenttime' will be passed from the main()

In [19]:
def simulation(numSeconds, pagesPerMinute):  # numSeconds(how long the printer will run) and the pages per minute
    
    labprinter = Printer(pagesPerMinute)
    printQueue = myQueue()
    waitingtimes = []
    
    # looping through the entire period while the printer is on
    for currentSecond in range(numSeconds):
        
        if newPrintTask(): # if there is a task in the queue which will be simulated little later
            task = Task(currentSecond)  # the task is defined which got created.
            printQueue.enqueue(task)  # the moment task got created, push it to the queue.
            
        # When the printer is not busy and there is a job in the queue    
        if (not labprinter.busy()) and (not printQueue.isEmpty()):
            nexttask = printQueue.dequeue()  # poping the task from the queue and giving to the printer
            waitingtimes.append(nexttask.waitTime(currentSecond))
            labprinter.startNext(nexttask)  # printing starts
            
        labprinter.tick()  # ticks the clock until the printing goes on
        
    averageWait = sum(waitingtimes) /  len(waitingtimes)
    print("Average Wait %6.2f secs %3d tasks remaining." %(averageWait, printQueue.size()))
    
def newPrintTask():
    num = random.randrange(1, 181)
    if num == 180:
        return True
    else:
        return False
        
for i in range(10):
    simulation(3600, 10)

Average Wait  79.00 secs   0 tasks remaining.
Average Wait  60.10 secs   0 tasks remaining.
Average Wait  20.73 secs   1 tasks remaining.
Average Wait  27.00 secs   1 tasks remaining.
Average Wait   1.13 secs   0 tasks remaining.
Average Wait  11.24 secs   0 tasks remaining.
Average Wait   4.57 secs   0 tasks remaining.
Average Wait  12.23 secs   1 tasks remaining.
Average Wait  15.90 secs   1 tasks remaining.
Average Wait   4.44 secs   1 tasks remaining.
